In [28]:
def check_syms_in_cell(Cell, matrixes):
    transformed_points = Cell @ matrixes 
    transformed_points_sorted = np.array([np.unique(points, axis=0) for points in transformed_points])
    return ~ np.array([np.array_equal(points, Cell) for points in transformed_points_sorted])

# %time check_syms_in_cell(SUPERCELL, MAT.all_matrixes)

In [29]:
# def pick_matrixes_by_index(masked_indexes, matrixes=MAT.all_matrixes):
#     return np.ma.masked_where(masked_indexes, matrixes).compressed()

In [30]:
def compare_points(p1, p2):
    for a, b in zip(p1, p2):
        if a != b:
            return False
    return True

In [31]:
def get_all_transformed_points(matrix, base_point):
    transformed_point = matrix @ base_point
    while not compare_points(transformed_point, base_point):
        yield transformed_point
        transformed_point = matrix @ transformed_point  

In [32]:
def get_matrixes_for_specific_point_group(symmetries):
    output = np.ma.masked_where(symmetries, MAT.all_labels)
    mymask = np.array([np.full((3,3), val) for val in output.mask])
    return MAT.pick_matrixes_by_index(mymask)

In [84]:
# @profiler("time")
def reduce_cell_by_symmetry(cell, matrixes):
    cell_dict = {tuple(point): i for i, point in enumerate(cell, 1)}
    for matrix in matrixes:
        for base_point in cell:
            if cell_dict[tuple(base_point)]:
                for point in get_all_transformed_points(matrix, base_point):
                    cell_dict[tuple(point)] = 0
            
    reduced_cell, reduced_cell_indexes = [el for el in zip(*((point, index) for point, index in cell_dict.items() if index))]
    reduced_cell, reduced_cell_indexes = np.array(reduced_cell), np.array(reduced_cell_indexes)
    reduced_cell_indexes -= 1
    return reduced_cell, reduced_cell_indexes

In [87]:
import numpy as np
import cifParsing as CPRS
import MMfunc as MM
import Matrixes_new as MAT
from Profiling import profiler

SUPERCELL, _ = CPRS.getSCell(1100043, size=10) #'cif files/ZnS-Sfaleryt.cif'
# SUPERCELL

In [60]:
_ = np.c_[SUPERCELL, np.ones(len(SUPERCELL))] #add ones to each point in SUPERCELL

In [37]:
len(SUPERCELL)

8631

In [88]:
SYM = check_syms_in_cell(SUPERCELL, MAT.all_matrixes)
matrixes_ZnS = get_matrixes_for_specific_point_group(SYM)
%time new_cell, new_cell_indexes = reduce_cell_by_symmetry(SUPERCELL, matrixes_ZnS)

new_cell,new_cell_indexes  

CPU times: user 478 ms, sys: 0 ns, total: 478 ms
Wall time: 479 ms


(array([[-1.  , -1.  , -1.  ],
        [-1.  , -1.  , -0.8 ],
        [-1.  , -1.  , -0.6 ],
        ...,
        [-0.1 , -0.1 ,  0.  ],
        [-0.05, -0.05,  0.05],
        [ 0.  ,  0.  ,  0.  ]]),
 array([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
          11,   12,   13,   14,   15,   16,   17,   18,   19,   20,   22,
          23,   24,   25,   26,   27,   28,   29,   30,   33,   34,   35,
          36,   37,   38,   39,   40,   44,   45,   46,   47,   48,   49,
          50,   55,   56,   57,   58,   59,   60,   66,   67,   68,   69,
          70,   77,   78,   79,   80,   88,   89,   90,   99,  100,  110,
         221,  222,  223,  224,  225,  226,  227,  228,  229,  230,  231,
         232,  233,  234,  235,  236,  237,  238,  239,  242,  243,  244,
         245,  246,  247,  248,  249,  252,  253,  254,  255,  256,  257,
         258,  263,  264,  265,  266,  267,  268,  273,  274,  275,  276,
         277,  284,  285,  286,  287,  294,  295,  296,  30

In [36]:
len(new_cell)

476

In [66]:
SUPERCELL

array([[-1.  , -1.  , -1.  ],
       [-1.  , -1.  ,  0.  ],
       [-1.  , -1.  ,  1.  ],
       [-1.  , -0.5 , -0.5 ],
       [-1.  , -0.5 ,  0.5 ],
       [-1.  ,  0.  , -1.  ],
       [-1.  ,  0.  ,  0.  ],
       [-1.  ,  0.  ,  1.  ],
       [-1.  ,  0.5 , -0.5 ],
       [-1.  ,  0.5 ,  0.5 ],
       [-1.  ,  1.  , -1.  ],
       [-1.  ,  1.  ,  0.  ],
       [-1.  ,  1.  ,  1.  ],
       [-0.75, -0.75, -0.75],
       [-0.75, -0.75,  0.25],
       [-0.75, -0.25, -0.25],
       [-0.75, -0.25,  0.75],
       [-0.75,  0.25, -0.75],
       [-0.75,  0.25,  0.25],
       [-0.75,  0.75, -0.25],
       [-0.75,  0.75,  0.75],
       [-0.5 , -1.  , -0.5 ],
       [-0.5 , -1.  ,  0.5 ],
       [-0.5 , -0.5 , -1.  ],
       [-0.5 , -0.5 ,  0.  ],
       [-0.5 , -0.5 ,  1.  ],
       [-0.5 ,  0.  , -0.5 ],
       [-0.5 ,  0.  ,  0.5 ],
       [-0.5 ,  0.5 , -1.  ],
       [-0.5 ,  0.5 ,  0.  ],
       [-0.5 ,  0.5 ,  1.  ],
       [-0.5 ,  1.  , -0.5 ],
       [-0.5 ,  1.  ,  0.5 ],
       [-0

In [76]:
from hashlib import sha1
import numpy as np

class Hashable:
    def __init__(self, wrapped, tight=False):
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(np.uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return np.all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        return np.array(self.__wrapped) if self.__tight else self.__wrapped
#         if self.__tight:
#             return array(self.__wrapped)
#         return self.__wrapped
#     def __call__(self):
#         return self.__wrapped
    
    def __mul__(self, other):
        return self.__wrapped @ other
    
#     def __rmul__(self, other):
#         return self.__wrapped @ other

hsh = Hashable(np.array([1,0,1]))
# hsh._Hashable__hash
# dd = dict()
# dd[hsh] = 1 
# dd.keys()
# hash(hsh)
hsh * np.array([[0,-1,0], [1,0,0], [0,0,-1]]) 

list(get_all_transformed_points_hash(np.array([[0,-1,0], [1,0,0], [0,0,-1]]) , hsh))

TypeError: 'Hashable' object is not iterable

In [35]:
def sil(num):
    if num < 2:
        return 1
    prod = 1
    for i in range(2, num + 1):
        prod *= i
    return prod    

In [58]:
def NchooseK(n, k):
    for ik in range(1, k):
        n *= n - ik
    return n // sil(k)

vacs = 2

NchooseK(476,vacs), NchooseK(8631,vacs)

(113050, 37242765)

In [8]:
def get_all_transformed_points_hash(matrix, base_point):
    transformed_point = base_point * matrix
    while not compare_points(transformed_point, base_point):
        yield transformed_point
        transformed_point = transformed_point * matrix

In [53]:
# @profiler("time")
def reduce_cell_by_symmetry(cell, matrixes):
    cell_copy = [Hashable(point) for point in cell]
    cell_dict = {Hashable(point): True for point in cell}
    for matrix in matrixes:
        for base_point in cell_copy:
            if cell_dict[base_point]:
                for point in get_all_transformed_points(matrix, base_point):
                    cell_dict[Hashable(point)] = False
    return np.array([point for point, v in cell_dict.items() if v])
#     return [point for point, v in cell_dict.items() if v]

%timeit -n 10 reduce_cell_by_symmetry(SUPERCELL, matrixes_ZnS)


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [2]:
Matrixes = generateSymetryBase()

oski1 = [Matrixes["m"][os] for os in Matrixes["m"]][3:] 
oski2 =  [Matrixes["-4"][os] for os in Matrixes["-4"]] + [Matrixes["3"][os] for os in Matrixes["3"]][1:]
oski = oski1 + oski2

In [3]:
def makeCellWithVacancies(cell,indexes):
    indexes = set(indexes)
    cellVac = []
    Vac = []
    for nr, p in enumerate(cell):
        if nr not in indexes:
            cellVac.append(p)
        else:
            Vac.append(p)
    return np.array(cellVac)#, np.array(Vac)

In [8]:
def usunkoor(koorZEW, oski):
    koorWEW = koorZEW.copy()
    for os in oski:
        j = 1
        while j < len(koorWEW):            
            for punktdous in listadous(os, koorWEW[j]):                
                i = findindex(punktdous, koorWEW.T[:,j:])                
                if i + 1 and porownajPunkty(koorWEW[i+j],punktdous):
                    koorWEW = np.delete(koorWEW, i+j, 0)
            j += 1
    return koorWEW

# kom = makeCellWithVacancies(SUPERCELL,(0,))
# print(kom)
# for n in range(2,15):
SUPERCELL = supercell(CELL, size = 1)
print(1)
%time lol = usunkoor(SUPERCELL, oski)
#     print(len(lol),len(SUPERCELL))
#     print()
print(round(len(lol)/len(SUPERCELL),6))
print()
# for el in lol:
#     print(el)

1
Wall time: 11 ms
0.277778



In [5]:
SUPERCELL = supercell(CELL, size = 2)
%time usunkoor(SUPERCELL, oski) 

Wall time: 35 ms
Wall time: 34 ms


array([[-1.  , -1.  , -1.  ],
       [-1.  , -1.  ,  0.  ],
       [-1.  , -1.  ,  1.  ],
       [-1.  , -0.5 , -0.5 ],
       [-1.  , -0.5 ,  0.5 ],
       [-1.  ,  0.  ,  0.  ],
       [-1.  ,  1.  ,  1.  ],
       [-0.75, -0.75, -0.75],
       [-0.75, -0.75,  0.25],
       [-0.75, -0.25, -0.25],
       [-0.5 , -0.5 ,  0.  ],
       [-0.25, -0.25,  0.25],
       [ 0.  ,  0.  ,  0.  ]])

In [ ]:
%timeit usunkoor_mod(SUPERCELL, sorted(lol2)[0])